In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
import dash

In [ ]:
# The refactored code is a functional app that accepts multiple user inputs 
# and generates a SQL query according to the tables, columns, and processes selected. 
# It’s meant to illustrate the process of constructing a query based on different inputs. 
# In a production environment, a similar function could accept inputs from a variety of sources, 
# not just manual user inputs (as is the case in this app). 

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Schoolytics"),

    html.Div([
        html.Div([
                dcc.Dropdown(
                value=None,
                id = 'columns_select',
                placeholder='Select Columns',
                options=[{'label':'Course ID', 'value': 'course_id'},
                         {'label':'Status', 'value': 'status'},
                        {'label':'Student ID', 'value': 'student_id'}],
                style={'width':'100%','textAlign':'center'},
                multi=True,
                ),
        ]),
        
        html.Div([
                dcc.Dropdown(
                value=None,
                id = 'function_select',
                placeholder='Select Function',
                options=[{'label':'SUM', 'value': 'sum'},
                         {'label':'AVERAGE', 'value': 'avg'},
                            {'label':'COUNT', 'value': 'count'}],
                multi=False,
                style={'width':'100%','textAlign':'center'}
                ),
        ]),
        
        html.Div([
                dcc.Dropdown(
                value=None,
                id = 'function_column_select',
                placeholder='Select Function Columns',
                options=[{'label':'Coursework ID', 'value': 'course_work_id'},
                         {'label':'Status', 'value': 'status'},
                        {'label':'Assigned Points', 'value': 'assigned_points'},
                        {'label':'Max Points', 'value': 'max_points'}],
                multi=True,
                style={'width':'100%','textAlign':'center'}
                ),
        ]),
    
        html.Div([
                dcc.Dropdown(
                value='no_join',
                id = 'join_select',
                placeholder='Select Join Type',
                options=[{'label':'No Join', 'value': 'no_join'},
                         {'label':'Left Join', 'value': 'left join'},
                         {'label':'Right Join', 'value': 'right join'},
                        {'label':'Inner Join', 'value': 'inner join'},
                        {'label':'Outer Join', 'value': 'outer join'}],
                multi=False,
                style={'width':'50%','textAlign':'center'},
                clearable=False,
                ),
            ]),
        html.Div([
                dcc.Dropdown(
                value='no_join',
                id = 'join_select_table',
                placeholder='Select Join Table',
                options=[{'label':'No Join', 'value': 'no_join'},
                         {'label':'Roster', 'value': 'roster'},
                         {'label':'Student', 'value': 'student'},
                         {'label':'Course', 'value': 'course'},
                        {'label':'Course Work', 'value': 'course_work'}],
                multi=False,
                style={'width':'50%','textAlign':'center'},
                clearable=False,
                ),
                ]),
            
        html.Div([
                dcc.Dropdown(
                value='no_join',
                id = 'join_select_column',
                placeholder='Select Join Column',
                options=[{'label':'No Join', 'value': 'no_join'},
                         {'label':'Course ID', 'value': 'course_id'},
                         {'label':'Status', 'value': 'status'},
                        {'label':'Student ID', 'value': 'student_id'}],
                multi=False,
                style={'width':'50%','textAlign':'center'},
                clearable=False,
                ),
                ]),    
        
        html.Div([
                dcc.Dropdown(
                value='any',
                id = 'work_status_select',
                placeholder='Select Work Status',
                options=[{'label':'Any Status', 'value': 'any'},
                         {'label':'NEW', 'value': 'NEW'},
                         {'label':'TURNED_IN', 'value': 'TURNED_IN'}],
                multi=False,
                style={'width':'100%','textAlign':'center'}
                ),
        ]),
        
        
        html.Div([
                dcc.Dropdown(
                value=None,
                id = 'grouping_column_select',
                placeholder='Select Grouping Column',
                options=[{'label':'Teacher ID', 'value': 'teacher_id'},
                         {'label':'Course ID', 'value': 'course_id'},
                         {'label':'Student ID', 'value': 'student_id'}],
                multi=False,
                style={'width':'100%','textAlign':'center'}
                ),
        ]),
        
    ],style={'width': '35%', 'display': 'inline-block'}),
    
    html.H2('Generated SQL Query'),
    html.H2('',id='sql_query'),
    
])

# Although this call back is creating a query using User input within this app, this is only used as an
# illustration tool. In a production enviroment, a similar function could accept inputs from a variety
# of sources and still effectively return a query.
@app.callback(
    dash.dependencies.Output('sql_query', 'children'),
    [dash.dependencies.Input('columns_select', 'value'),
     dash.dependencies.Input('function_column_select', 'value'),
    dash.dependencies.Input('function_select', 'value'),
    dash.dependencies.Input('grouping_column_select', 'value'),
    dash.dependencies.Input('work_status_select', 'value'),
    dash.dependencies.Input('join_select', 'value'),
    dash.dependencies.Input('join_select_table', 'value'),
    dash.dependencies.Input('join_select_column', 'value')])
def generate_query(columns,func_cols,function,grp_col,work_status,join,j_table,j_column):
    try: 
        query='select '
        for col in columns:
            query+=str(col)
            query+=', '
        
        if function!=None:
            if function=='avg' and len(func_cols)==2:
                query+=str(function)
                query+='('
                query+=str(func_cols[0])
                query+='/'
                query+=str(func_cols[1])
                query+=')'
            else:
                for fcol in func_cols:
                    query+=str(function)
                    query+='('
                    query+=str(fcol)
                    query+='), '
            
        else:
            for fcol in func_cols:
                query+=str(fcol)
                query+=')'
        
        query+=' from student_submission '

        if join!='no_join' and j_table!='no_join' and j_column!='no_join':
            query+=str(join)
            query+=' '+str(j_table)+' on '
            query+='student_submission.'+str(j_column)+'='+str(j_table)+'.'+str(j_column)+' '
        
        if work_status!='any' and ('status' in columns or 'status' in func_cols):
            query+= 'where status = '+str(work_status)+' '
            
        if grp_col!=None:
            query+='group by '+str(grp_col)

        
    except TypeError:
        query='Check Inputs'
    return query

app.run_server(mode='external')